# Scenario Description 


There exists a lot real-world autonomous driving datasets, such as Waymo and nuPlan.

Those datasets contain rich information about road map, traffic lights, traffic participants and their motions.

Those data can be imported to the MetaDrive so that we can reconstruct an interactive environments and replay (or even alter!) those scenarios recorded in the real world.

To achieve that, we have a separated project called ScenarioNet (https://metadriverse.github.io/scenarionet/) that converts various sources of AD data into a unified data structure, manage those data, and import those data into MetaDrive for creating interactive environments.

As you can see in the next figure of ScenarioNet, the key functionalities developed in ScenarioNet are:

* **Unified Scenario Description**
* **Scenario Data Management**
* **Real-world Scenario Simulation**

 
<img src="figs/scenarionet.png" width="800">


MetaDrive implements the unified scenario description in `metadrive/scenario/scenario_description.py`. In this notebook, we will have a quick walkthrough of the data structure of the scenario description.

## What is Scenario Description?

**TL;DR: Scenario Description is a nested python dict that stores the information of one AD scenario.**

By defining the unified data structure, from the input end, we can define a set of toolkits to convert each dataset into the dataset in the form of Scenario Description.

Then we can store each scenario into a `.pkl` file in the local file system and manage those files with [ScenarioNet](https://github.com/metadriverse/scenarionet).

From the output end, no matter where the source data is from, we can reconstruct interactive environment via MetaDrive and conduct downstream tasks like RL training by reading from the dataset containing those `.pkl` files.

Here is an example of what the Scenario Description looks like:


```python
scenario = {

    # ===== Meta data about the scenario =====
    # string. The name of the scenario
    "id": "Waymo-001",

    # string. The version of data format.
    "version": "MetaDrive v0.3.0.1",

    # int. The length of all trajectory and state arrays (T).
    "length": 200,

    # ===== Meta data ===
    "metadata": {

        # np.ndarray in (T, ). The time stamp of each time step.
        "ts": np.array([0.0, 0.1, 0.2, ...], dtype=np.float32),


        # bool. Whether the scenario is processed and exported by MetaDrive.
        # Some operations may be done, such as interpolating the lane to
        # make way points uniformly scattered in given interval.
        "metadrive_processed": True,

        # string. Coordinate system.
        "coordinate": "metadrive",

        # optional keys
        "source_file": "training_20s.tfrecord-00014-of-01000",
        "dataset": "waymo",
        "scenario_id": "dd0c8c27fdd6ef59",  # Used in Waymo dataset
        "seed": 512,
        "history_metadata": {},

        "sdc_id": "172",  # A key exists in tracks

    },

    # ===== Trajectories of active participants, e.g. vehicles, pedestrians =====
    # a dict mapping object ID to it's state dict.
    "tracks": {
        "vehicle1": {

            # The type string in metadrive.scenario.MetaDriveType
            "type": "VEHICLE",

            # The state dict. All values must have T elements.
            "state": {
                "position": np.ones([200, 3], dtype=np.float32),
                ...
            },

            # The meta data dict. Store useful information about the object. type in metadata could be those from
            # different dataset
            "metadata": {
                "type": "VEHICLE",
                "track_length": 200,
                "object_id": "vehicle1",

                # Optional keys
                "agent_name": "default_agent",
                "policy_spawn_info": {  # Information needed to re-instantiate the policy
                    "policy_class": ("metadrive.policy.idm_policy", "IDMPolicy"),
                    "args": ...,
                    "kwargs": ...,
                }
            }
        },

        "pedestrian1": ...
    },

    # ===== States sequence of dynamics objects, e.g. traffic light =====
    # a dict mapping object ID to it's state dict.
    "dynamic_map_states": {
        "trafficlight1": {

            # The type string in metadrive.scenario.MetaDriveType
            "type": "TRAFFIC_LIGHT",

            # The state dict. All values must have T elements.
            "state": {
                "object_state": np.ones([200, ], dtype=int),
                ...
            },

            # The meta data dict. Store useful information about the object
            "metadata": {
                "type": "TRAFFIC_LIGHT",
                "track_length": 200,
            }
    }

    # ===== Map features =====
    # A dict mapping from map feature ID to a line segment
    "map_features": {
        "219": {
            "type": "LANE_SURFACE_STREET",
            "polyline": np.array in [21, 2],  # A set of 2D points describing a line segment
            # optional, only works for lane
            "polygon": np.array in [N, 2] # A set of 2D points representing convexhull
        },
        "182": ...
        ...
    }
}

# Note: Example is from the docstring of metadrive/scenario/scenario_description.py
```